In [1]:
import dicom # for reading dicom files
import os # for doing directory operations 
import pandas as pd # for some simple data analysis (right now, just to load in the labels data and quickly reference it)
import cv2
import numpy as np

In [2]:
# Change this to wherever you are storing your data:
# IF YOU ARE FOLLOWING ON KAGGLE, YOU CAN ONLY PLAY WITH THE SAMPLE DATA, WHICH IS MUCH SMALLER

data_dir = '/media/ai-master/my_passport/joyce/Sample Image/'
patients = os.listdir(data_dir)
labels_df = pd.read_csv('/media/ai-master/my_passport/joyce/stage1_labels.csv', index_col=0)

In [3]:
import numpy as np
import pandas as pd
import dicom
import os
import matplotlib.pyplot as plt
import cv2
import math

IMG_SIZE_PX = 50
SLICE_COUNT = 20

def chunks( l,n ):
    count=0
    for i in range(0, len(l), n):
        if(count < HM_SLICES):
            yield l[i:i + n]
            count=count+1


def mean(a):
    return sum(a) / len(a)


def process_data(patient,labels_df,img_px_size=50, hm_slices=20, visualize=False):
    
    label = labels_df.get_value(patient, 'cancer')
    path = data_dir + patient
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))

    new_slices = []
    slices = [cv2.resize(np.array(each_slice.pixel_array),(img_px_size,img_px_size)) for each_slice in slices]
    
    chunk_sizes = int(math.floor(len(slices) / HM_SLICES))
    for slice_chunk in chunks(slices, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)

    if len(new_slices) == hm_slices-1:
        new_slices.append(new_slices[-1])

    if len(new_slices) == hm_slices-2:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])

    if len(new_slices) > hm_slices:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val
        
    if len(new_slices) == hm_slices+1:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val

    if visualize:
        fig = plt.figure()
        for num,each_slice in enumerate(new_slices):
            y = fig.add_subplot(4,5,num+1)
            y.imshow(each_slice, cmap='gray')
        plt.show()

    if label == 1: label=np.array([0,1])
    elif label == 0: label=np.array([1,0])
        
    return np.array(new_slices),label


In [4]:
#                                               stage 1 for real.
path = '/media/ai-master/my_passport/joyce/pre_3d/'
data_dir = '/media/ai-master/my_passport/joyce/stage1/'
patients = os.listdir(data_dir)
labels = pd.read_csv('/media/ai-master/my_passport/joyce/stage1_labels.csv', index_col=0)

much_data = []
for num,patient in enumerate(patients):
    if num % 100 == 0:
        print(num)
    try:
        img_data,label = process_data(patient,labels,img_px_size=IMG_SIZE_PX, hm_slices=SLICE_COUNT)
        #print(img_data.shape,label)
        much_data.append([img_data,label])
    except KeyError as e:
        print('This is unlabeled data!')

np.save('muchdata-{}-{}-{}.npy'.format(IMG_SIZE_PX,IMG_SIZE_PX,SLICE_COUNT), much_data)

0


NameError: global name 'HM_SLICES' is not defined

In [5]:


import tensorflow as tf
import numpy as np

IMG_SIZE_PX = 50
SLICE_COUNT = 20

n_classes = 2
batch_size = 10

x = tf.placeholder('float')
y = tf.placeholder('float')

keep_rate = 0.8



In [6]:


def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    #                        size of window         movement of window as you slide about
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')



In [7]:
def convolutional_neural_network(x):
    #                # 5 x 5 x 5 patches, 1 channel, 32 features to compute.
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,32])),
               #       5 x 5 x 5 patches, 32 channels, 64 features to compute.
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,32,64])),
               #                                  64 features
               'W_fc':tf.Variable(tf.random_normal([54080,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    #                            image X      image Y        image Z
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)


    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)

    fc = tf.reshape(conv2,[-1, 54080])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output

In [8]:
much_data = np.load('muchdata-50-50-20.npy')
# If you are working with the basic sample data, use maybe 2 instead of 100 here... you don't have enough data to really do this
train_data = much_data[:-100]
validation_data = much_data[-100:]



prediction = convolutional_neural_network(x)
cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
optimizer = tf.train.AdamOptimizer(learning_rate=10**-3).minimize(cost)
a = tf.cast(tf.argmax(prediction, 1),tf.float32)
b = tf.cast(tf.argmax(y,1),tf.float32)
auc = tf.contrib.metrics.streaming_auc(a, b)
hm_epochs = 50
saver = tf.train.Saver()

In [12]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.initialize_local_variables()) # try commenting this line and you'll get the error
    successful_runs = 0
    total_runs = 0

    for epoch in range(hm_epochs):
        epoch_loss = 0
        for data in train_data:
            total_runs += 1
            try:
                X = data[0]
                Y = data[1]
                _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                epoch_loss += c
                successful_runs += 1
            except Exception as e:
                # I am passing for the sake of notebook space, but we are getting 1 shaping issue from one 
                # input tensor. Not sure why, will have to look into it. Guessing it's
                # one of the depths that doesn't come to 20.
                pass
                #print(str(e))

        print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss/len(train_data))

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

        print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))

#         pred=tf.argmax(prediction,1)
#         pred.eval(feed_dict={x: [i[0] for i in validation_data]})
#         
        pred = tf.argmax(prediction, 1).eval({x:[i[0] for i in validation_data]})

#         tf.metrics.auc(label, pred)
    label = [i[1] for i in validation_data]
    auc = tf.metrics.auc(labels, pred, weights=None, 
                         num_thresholds=200, metrics_collections=None, updates_collections=None, 
                         curve='ROC', name=None) 
    save_path = saver.save(sess, "temp/model4.ckpt")
    print('auc:',auc)
    print('Done. Finishing accuracy:')
    print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
    
    print('fitment percent:',successful_runs/total_runs)

# Run this locally:
#train_neural_network(x)

Instructions for updating:
Use `tf.local_variables_initializer` instead.
('Epoch', 1, 'completed out of', 50, 'loss:', 201894034.10832691)
('Accuracy:', 0.65999997)
('Epoch', 2, 'completed out of', 50, 'loss:', 15797974.982074017)
('Accuracy:', 0.69)
('Epoch', 3, 'completed out of', 50, 'loss:', 5776794.2952968385)
('Accuracy:', 0.58999991)
('Epoch', 4, 'completed out of', 50, 'loss:', 3306051.7332787202)
('Accuracy:', 0.47999999)
('Epoch', 5, 'completed out of', 50, 'loss:', 1735958.5535008674)
('Accuracy:', 0.66999996)
('Epoch', 6, 'completed out of', 50, 'loss:', 983361.15527720226)
('Accuracy:', 0.57999998)
('Epoch', 7, 'completed out of', 50, 'loss:', 694363.34969370416)
('Accuracy:', 0.58999997)
('Epoch', 8, 'completed out of', 50, 'loss:', 392055.49649048294)
('Accuracy:', 0.52999997)
('Epoch', 9, 'completed out of', 50, 'loss:', 206870.49961786365)
('Accuracy:', 0.66999996)
('Epoch', 10, 'completed out of', 50, 'loss:', 125682.17820550909)
('Accuracy:', 0.48999998)
('Epoch', 11

ValueError: Argument must be a dense tensor:                                   cancer
id                                      
0015ceb851d7251b8f399e39779d1e7d       1
0030a160d58723ff36d73f41b170ec21       0
003f41c78e6acfa92430a057ac0b306e       0
006b96310a37b36cccb2ab48d10b49a3       1
008464bb8521d09a42985dd8add3d0d2       1
0092c13f9e00a3717fdc940641f00015       0
00986bebc45e12038ef0ce3e9962b51a       0
00cba091fa4ad62cc3200a657aeb957e       0
00edff4f51a893d80dae2d42a7f45ad1       1
0121c2845f2b7df060945b072b2515d7       0
013395589c01aa01f8df81d80fb0e2b8       0
01de8323fa065a8963533c4a86f2f6c1       0
01e349d34c06410e1da273add27be25c       0
01f1140c8e951e2a921b61c9a7e782c2       0
024efb7a1e67dc820eb61cbdaa090166       0
0257df465d9e4150adef13303433ff1e       1
0268f3a7a17412178cfb039e71799a80       0
026be5d5e652b6a7488669d884ebe297       0
02801e3bbcc6966cb115a962012c35df       1
028996723faa7840bb57f57e28275e4c       1
0334c8242ce7ee1a6c1263096e4cc535       0
03fb0d0fdb18

In [47]:

correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
with tf.Session() as sess:
    
    saver.restore(sess, "temp/model3.ckpt")

    print('Validation Accuracy:',
          accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))

#     sess.close()
#     print('training Accuracy:',
#           accuracy.eval({x:[i[0] for i in train_data], y:[i[1] for i in train_data]}))
    print pred

('Validation Accuracy:', 0.71999997)
Tensor("ArgMax_338:0", shape=(?,), dtype=int64)


In [30]:
print predictions

[array([[  4.81511328e+08,  -3.50462336e+08]], dtype=float32), array([[  4.33265760e+08,   2.67488112e+08]], dtype=float32), array([[  4.02388576e+08,  -9.18245056e+08]], dtype=float32), array([[  8.62159808e+08,  -1.67334413e+09]], dtype=float32), array([[ -4.82582592e+08,  -5.95553408e+08]], dtype=float32), array([[ -4.81497440e+08,   2.04550848e+08]], dtype=float32), array([[ -2.70466944e+08,  -5.56810368e+08]], dtype=float32), array([[  7.39532608e+08,  -2.61643664e+08]], dtype=float32), array([[  3.29593856e+08,  -2.73537152e+08]], dtype=float32), array([[  1.32420557e+09,   3.34509632e+08]], dtype=float32), array([[  2.46718352e+08,  -1.09019443e+09]], dtype=float32), array([[  9.86950912e+08,  -4.29186272e+08]], dtype=float32), array([[  7.13097856e+08,   3.11522944e+08]], dtype=float32), array([[  9.30033664e+08,  -1.16569536e+08]], dtype=float32), array([[  8.25865600e+07,  -4.01317248e+08]], dtype=float32), array([[  7.65082304e+08,  -4.20561760e+08]], dtype=float32), array([

In [33]:
745.5101/1297
1-0.72

0.28